#Word2Box

Author: Bigarella Chiara

# 1 - Introduction

 The aim of this notebook is to provide an in-depth analysis of the **word2box** algorithm presented in [Word2Box: Capturing Set-Theoretic Semantics of Words using Box Embeddings](https://arxiv.org/abs/2106.14361) (2022, Dasgupta et al.).

 The original code is freely available at the following GitHub repository: https://github.com/iesl/word2box



## 1.1 Project Structure




The *word2box* project has the following folder structure. In **section 3** I will go through all the modules and I will explain the behavior of all the classes and functions defined in the project.

```
word2box/
  data/
    ptb/
    similarity_datasets/
  src/
    language_modeling_with_boxes/
      box/
        box_wrapper.py
        modules.py
        utils.py
      datasets/
        utils.py
        word2vecgpu.py
      models/
        BaseModule.py
        word2box.py
      train/
        loss.py
        negative_sampling.py
        train.py
        Trainer.py
```

#2 - Setup

In this section, we install and import all the dependencies needed to run the project. We also create some constants and add some global settings.

## 2.1 Install dependencies

Before installing the dependencies, we have to uninstall some modules to avoid dependency conflicts due to wrong versions. After that, we can install the required packages.

In [ ]:
# uninstall the following modules to avoid dependency conflicts

! pip uninstall torch -y
! pip uninstall torchtext -y
! pip uninstall torchaudio -y
! pip uninstall torchdata -y
! pip uninstall torchsummary -y
! pip uninstall torchvision -y
! pip uninstall fastai -y

In [ ]:
! pip install click>=7.0
! pip install attrs
! pip install toml
! pip install torchtext==0.6.0
! pip install tqdm
! pip install xopen
! pip install loguru
! pip install typer[all]
! pip install wandb==0.10.33
! pip install wandb-utils

To install the `pytorch_utils` package, we need to clone the **word2box** repository and install the package from the local folder.

In [ ]:
%cd /content
! rm -rf word2box

! git clone https://github.com/ChiaraBi/word2box.git --recurse-submodules

%cd /content/word2box/lib/pytorch-utils
! pip install .

## 2.2 Imports

In [ ]:
import argparse
import attr
import csv
import itertools
import json
import math
from math import ceil
import os
import pickle
import pprint
import random

from multiprocessing import Manager
from pathlib import Path
from tqdm import tqdm
from typing import *
import wandb
from xopen import xopen

from pytorch_utils import TensorDataLoader

import torchtext, torch
from torch import LongTensor, BoolTensor, Tensor
from torch.autograd import Variable
import torch.nn as nn
from torch.utils.data import Dataset

import numpy as np
from scipy import special
from scipy.stats import spearmanr

## 2.3 Constants and settings

In [ ]:
tanh_eps = 1e-20
euler_gamma = 0.57721566490153286060

In [ ]:
global use_cuda
use_cuda = torch.cuda.is_available()
device = torch.cuda.current_device() if use_cuda else "cpu"

#3 - Modules

Here I present the **word2box** classes and functions and, for each one of them, I provide a description of their behaviour.

I considered only the code needed for training and testing the `Word2BoxConjuction` model, since it is the only model for which the original code works. For each piece of code, I point to the codebase file it came from.

There might be small differences with the original codebase, since I fixed some bugs and removed unecessary or not working code.

## 3.1 datasets

### Word2VecDatasetOnDevice

The `Word2VecDatasetOnDevice` class inherits from `torch.utils.dataDataset`.

The **constructor** takes in input the following parameters:
* `corpus: LongTensor`: a chunk of text from the dataset.
* `device: Union[int, str]`: `'cpu'` or `'gpu'`.
* `eos_mask: bool`: tells whether to use or not an end of sentence (EOS) mask, i.e. a boolean mask used to indicate the positions of the end of sentences in a batch of token sequences. By default it is set to `True`.
* `subsample_thresh: float`: the subsample treshold. By default it is set to `1e-3`.
* `vocab: Union[Dict, Any]`: an instance of `torchtext.data.Field()`. This object has two main attributes: `stoi` (string-to-index) and `itos` (index-to-string).
`vocab.stoi` is a dictionary that maps each unique word in the vocabulary to a unique index. This is useful for converting a sequence of words into a sequence of indices, which can then be input to a model. `vocab.itos` is a list where the element at each index is the corresponding word. This is useful for converting a sequence of indices back into a sequence of words.
* `window_size: int`: the window size. By default it is set to `10`.

It sets the `pad_id`, `eos_token`, and `pad_size` and computes the `subsampling_prob`.

**Methods**

* `__getitem__(idx: LongTensor)`: this method takes in input the indices of the center words in the corpus. Then, it performs the following actions:
  1. For each center word index, it computes the corresponding context word indices.
  2. It perform the subsampling for both center and context words by calling the `sub_sample_words` method.
  3. It gets rid of the data that has `<pad>` as the center word or that has all context words as `<pad>`.
  4. It returns a dictionary with `center_word`, `context_words` and `context_mask`.

* `__len__()`: returns the length of the corpus minus the padding.
* `to(device: Union[torch.device, str])`: takes in input the `device` and returns an instance of `Word2VecDatasetOnDevice` where the corpus is `self.corpus.to(device)`.
* `sub_sample_words(_input: LongTensor)`: masks out the subsampled words by replacing them with `pad_id`.
* `get_mask(_input: LongTensor)`: returns a mask for the context words that have the `<pad>` token.

In [ ]:
# datasets/word2vecgpu.py

class Word2VecDatasetOnDevice(Dataset):
    def __init__(
        self,
        corpus: LongTensor,
        window_size: int = 10,
        vocab: Union[Dict, Any] = None,
        subsample_thresh: float = 1e-3,
        eos_mask: bool = True,
        device: Union[int, str] = None,
    ):
        self.corpus = corpus
        self.window_size = window_size
        self.vocab = vocab
        self.subsample_thresh = subsample_thresh
        self.eos_mask = eos_mask
        self.pad_id = torch.tensor(self.vocab.stoi["<pad>"]).to(self.corpus.device)
        self.eos_token = torch.tensor(self.vocab.stoi["<eos>"]).to(self.corpus.device)
        # pad this at the beginning and end with window_size number of padding
        self.pad_size = 10
        total_words = sum(self.vocab.freqs.values())
        unigram_prob = (
            torch.tensor([self.vocab.freqs.get(key, 0) for key in self.vocab.itos])
            / total_words
        )
        self.subsampling_prob = 1.0 - torch.sqrt(
            subsample_thresh / (unigram_prob + 1e-19)
        ).to(
            self.corpus.device
        )  #

    def __getitem__(
        self, idx: LongTensor
    ) -> Tuple[LongTensor, LongTensor, BoolTensor, BoolTensor]:
        # idx is a Tensor of indicies of the corpus, eg. [2342,12312312,34534,1]
        # we will interpret these as the id of the center word
        idx += self.pad_size
        # Idx is repeated to get the sliding window effect
        # For the sliding window part we add the range with idx
        window_range = torch.arange(-self.window_size, self.window_size + 1)
        idx = idx.unsqueeze(1) + window_range.unsqueeze(0)

        # idx = torch.transpose(idx.repeat(2*self.window_size+1,1), 0, 1)
        # idx = idx + torch.arange(-self.window_size, self.window_size+1)

        # Get the middle slice for the center
        # The rest of them are context
        center = self.corpus[idx[:, self.window_size]]
        context = self.corpus[
            torch.cat(
                (idx[:, : self.window_size], idx[:, self.window_size + 1 :]), dim=1
            )
        ]
        # Get do the subsampling.
        center = self.sub_sample_words(center)
        context = self.sub_sample_words(context)

        # Get rid of the dataset that has the center word as <pad>.
        # Or has all context words as <pad>.
        if not self.eos_mask:
            keep = (center != self.pad_id) & (context != self.pad_id).any(dim=-1)
            center = center[keep]
            context = context[keep]
            assert (center != self.pad_id).all()
            context_mask = torch.ones_like(context)
        else:
            keep = (
                (center != self.pad_id)
                & (context != self.pad_id).any(dim=-1)
                & (center != self.eos_token)
                & (context != self.eos_token).any(dim=-1)
            )
            center = center[keep]
            context = context[keep]
            assert (center != self.pad_id).all()
            context_mask = self.get_mask(context)
            # Mask might do away with the whole sentence. In that case remove that
            keep = (context_mask != False).any(dim=1).squeeze()
            center = center[keep]
            context = context[keep]
            context_mask = context_mask[keep]
        return {
            "center_word": center,
            "context_words": context,
            "context_mask": context_mask,
        }

    def __len__(self) -> int:
        return len(self.corpus) - 2 * self.pad_size

    def to(self, device: Union[torch.device, str]):
        return Word2VecDatasetOnDevice(
            self.corpus.to(device), self.window_size, self.vocab, self.subsample_thresh
        )

    def sub_sample_words(self, _input: LongTensor) -> BoolTensor:
        ## Mask out the subsampled words. We will do so by
        ## replacing them with pad ids.
        mask_prob = torch.rand(_input.shape).to(_input.device)
        _input[mask_prob < self.subsampling_prob[_input]] = self.pad_id
        return _input

    def get_mask(self, _input: LongTensor) -> BoolTensor:
        ## Get the mask for the contexts that has pad token.
        right_mask = ~(
            (_input[:, self.window_size :] == self.eos_token).cumsum(dim=-1) > 0
        )
        l_eos = _input[:, : self.window_size] == self.eos_token
        left_mask = ~(
            l_eos | (l_eos.any(dim=-1, keepdim=True) & (l_eos.cumsum(dim=-1) == 0))
        )
        mask = torch.cat((left_mask, right_mask), dim=-1)
        return (_input != self.pad_id) & mask


### LazyDatasetLoader

The `LazyDatasetLoader` class represents a data loader. The class decorator `@attr.s` adds dunder methods to the class and the parameter `auto_attribs=True`collects PEP 526-annotated attributes from the class body.

**Methods**

* `__attrs_post_init__()`: computes the length and the `training_tensor_chunks`.
* `__iter__()`: iterates over `training_tensor_chunks`. For each chunk, it creates an instance of `Word2VecDatasetOnDevice` and then it yields an iterator to the dataset chunk.
* `__len__()`: returns the length computed in `__attrs_post_init__`.

In [ ]:
# datasets/word2vecgpu.py

@attr.s(auto_attribs=True)
class LazyDatasetLoader:
    training_tensor: Tensor
    n_splits: int
    window_size: int = 10
    vocab: Union[Dict, Any] = None
    subsample_thresh: float = 1e-3
    eos_mask: bool = True
    batch_size: int = 64
    device: Union[int, str] = None

    def __attrs_post_init__(self):
        lng = len(self.training_tensor)
        splits = [int(lng / 10) * i for i in range(10)]
        splits.append(lng)
        self.leng = sum(
            ceil((j - i) / self.batch_size) for i, j in zip(splits[:-1], splits[1:])
        )
        self.training_tensor_chunks = [
            self.training_tensor[splits[i] : splits[i + 1]] for i in range(10)
        ]

    def __iter__(self):
        for chunk in self.training_tensor_chunks:
            train_dataset = Word2VecDatasetOnDevice(
                corpus=chunk,
                window_size=self.window_size,
                vocab=self.vocab,
                subsample_thresh=self.subsample_thresh,
                eos_mask=self.eos_mask,
            ).to(self.device)
            train_iter = TensorDataLoader(train_dataset, self.batch_size, shuffle=True)
            yield from train_iter
            del train_dataset
            del train_iter

    def __len__(self):
        return self.leng

### utils

* `load_vocab(data_dir: Union[str, Path])`: takes in input the data directory (`data_dir`) and returns the vocabulary numerical identifiers and frequencies.
* `load_tokenizer(data_dir)`: takes in input the data directory and returns the tokenized training dataset.
* `load_train_data_as_tensor(data_dir)`: takes in input the data directory and returns the training dataset as a `torch.Tensor`.
* `get_iter_on_device`:
  - input parameters:
    - `batch_size`: batch size.
    - `dataset`: dataset name. It should be the name of the corresponding dataset folder inside the `data` folder.
    - `data_device`: `'cpu'` or `'gpu'`.
    - `eos_mask: bool`: tells whether to use or not an End Of Sentence (EOS) mask, i.e. a boolean mask used to indicate the positions of the end of sentences in a batch of token sequences.
    - `n_gram`: window size.
    - `subsample_thresh`: the subsample treshold.
  - returns the vocabulary statistics (`TEXT`) and an instance of `LazyDatasetLoader` for the training data.

In [ ]:
# datasets/utils.py

def load_vocab(data_dir: Union[str, Path]):
    vocab_tsv = Path(data_dir + "vocab.tsv")
    vocab_stoi = {}
    vocab_freq = {}
    if vocab_tsv.exists():
        with vocab_tsv.open() as vocab_file:
            next(vocab_file)  # skips header line
            for token_id, line in enumerate(vocab_file):
                token, frequency = line.split()
                vocab_stoi[token] = int(token_id)
                vocab_freq[token] = int(frequency)
    elif os.path.isfile(data_dir + "vocab_stoi.json") and os.path.isfile(
        data_dir + "vocab_freq.json"
    ):
        vocab_stoi = json.load(open(data_dir + "vocab_stoi.json", "r"))
        vocab_freq = json.load(open(data_dir + "vocab_freq.json", "r"))
    else:
        TEXT = torchtext.data.Field()
        train_split = torchtext.datasets.LanguageModelingDataset.splits(
            path=data_dir,
            train="train.txt",
            validation=None,
            test=None,
            text_field=TEXT,
        )
        TEXT.build_vocab(train_split[0])
        vocab_stoi_file = open(data_dir + "vocab_stoi.json", "a")
        vocab_freq_file = open(data_dir + "vocab_freq.json", "a")
        json.dump(TEXT.vocab.stoi, vocab_stoi_file)
        json.dump(TEXT.vocab.freqs, vocab_freq_file)
        vocab_stoi_file.close()
        vocab_freq_file.close()
        vocab_stoi = TEXT.vocab.stoi
        vocab_freq = TEXT.vocab.freqs
    return vocab_stoi, vocab_freq


def load_tokenizer(data_dir):
    if os.path.isfile(data_dir + "train_tokenized.pkl"):
        train_tokenized = pickle.load(open(data_dir + "train_tokenized.pkl", "rb"))
    else:
        train_tokenized = []
        vocab_stoi = json.load(open(data_dir + "vocab_stoi.json", "r"))
        vocab_freq = json.load(open(data_dir + "vocab_freq.json", "r"))

        with open(data_dir + "train.txt", "r") as f:
            for line in f:
                words = line.split()
                train_tokenized.append(
                    [vocab_stoi[ele] for ele in words] + [vocab_stoi["<eos>"]]
                )

        pickle.dump(train_tokenized, open(data_dir + "train_tokenized.pkl", "wb"))
    return train_tokenized


def load_train_data_as_tensor(data_dir):
    tensor_file = Path(data_dir + "train.pt")
    if tensor_file.exists():
        return torch.load(tensor_file)
    else:
        train_tensor = torch.tensor(
            list(itertools.chain.from_iterable(load_tokenizer(data_dir)))
        )
        torch.save(train_tensor, tensor_file)
    return train_tensor


def get_iter_on_device(
    batch_size,
    dataset,
    n_gram,
    subsample_thresh,
    data_device,
    eos_mask,
):
    print("Loading VOCAB & Tokenized Training files ...")

    data_dir = "./data/" + dataset + "/"
    vocab_stoi, vocab_freq = load_vocab(data_dir)
    train_tokenized = load_train_data_as_tensor(data_dir)

    ## Create Vocabulary properties
    print("Creating iterable dataset ...")
    TEXT = torchtext.data.Field()
    TEXT.stoi = vocab_stoi
    TEXT.freqs = vocab_freq
    TEXT.itos = [k for k, v in sorted(vocab_stoi.items(), key=lambda item: item[1])]

    # Since we won't train on <pad> and <eos>. These should not come in any sort of
    # subsampling and negative sampling part.
    TEXT.freqs["<pad>"] = 0
    TEXT.freqs["<unk>"] = 0

    if eos_mask:
        TEXT.freqs["<eos>"] = 0

    ## Create data on the device
    print("Creating iterable dataset on GPU/CPU...")
    if data_device == "gpu":
        data_device = device
    train_iter = LazyDatasetLoader(
        training_tensor=train_tokenized,
        n_splits=1000,
        window_size=n_gram,
        vocab=TEXT,
        subsample_thresh=subsample_thresh,
        eos_mask=eos_mask,
        device=data_device,
        batch_size=batch_size,
    )

    return TEXT, train_iter


## 3.2 box

### utils

The `log1mexp` function computes the logarithm of the complementary exponential function. In mathematical terms, it calculates `log(1 - exp(x))`.

This function is useful in numerical computations because it can provide more accurate results when `x` is close to zero. Directly computing `log(1 - exp(x))` can result in a loss of precision due to the subtraction in the argument of the logarithm. The `log1mexp` function is designed to avoid this issue.

In [ ]:
# box/utils.py

_log1mexp_switch = math.log(0.5)


def log1mexp(
    x: torch.Tensor, split_point=_log1mexp_switch, exp_zero_eps=1e-7
) -> torch.Tensor:
    """
    Computes log(1 - exp(x)).

    Splits at x=log(1/2) for x in (-inf, 0] i.e. at -x=log(2) for -x in [0, inf).

    = log1p(-exp(x)) when x <= log(1/2)
    or
    = log(-expm1(x)) when log(1/2) < x <= 0

    For details, see

    https://cran.r-project.org/web/packages/Rmpfr/vignettes/log1mexp-note.pdf

    https://github.com/visinf/n3net/commit/31968bd49c7d638cef5f5656eb62793c46b41d76
    """
    logexpm1_switch = x > split_point
    Z = torch.zeros_like(x)
    # this clamp is necessary because expm1(log_p) will give zero when log_p=1,
    # ie. p=1
    logexpm1 = torch.log((-torch.expm1(x[logexpm1_switch])).clamp_min(1e-38))
    # hack the backward pass
    # if expm1(x) gets very close to zero, then the grad log() will produce inf
    # and inf*0 = nan. Hence clip the grad so that it does not produce inf
    logexpm1_bw = torch.log(-torch.expm1(x[logexpm1_switch]) + exp_zero_eps)
    Z[logexpm1_switch] = logexpm1.detach() + (logexpm1_bw - logexpm1_bw.detach())
    # Z[1 - logexpm1_switch] = torch.log1p(-torch.exp(x[1 - logexpm1_switch]))
    Z[~logexpm1_switch] = torch.log1p(-torch.exp(x[~logexpm1_switch]))

    return Z


### BoxTensor

The `BoxTensor` class is a wrapper for a `torch.Tensor` and represents single or multiple boxes.

The **constructor** takes in input the following parameters and checks whether the tensor shape is correct:
* `data: torch.Tensor`: a Tensor that represents our box or boxes.
* `learnt_temp: bool`: tells whether to use a learned temperature parameter, which is often used in models that employ a softmax function for probabilistic predictions. By default it is set to `False`.

`BoxTensor` exposes the following methods and properties.

**Properties**

* `z: torch.Tensor` : lower left coordinates as Tensor.
* `Z: torch.Tensor` : top right coordinates as Tensor.
* `box_type: String`: the class name.
* `centre: torch.Tensor`: centre coordinates as Tensor.

**Class methods**

* `from_zZ`:
  1. checks if `z` and `Z` have the same shape. If not, it throws an error.
  2. creates a `torch.Tensor` by concatenating `z` and `Z`.
  4. wraps the torch tensor with a `BoxTensor` and returns it.
* `from_split`: creates a `BoxTensor` by splitting a `torch.Tensor` on the dimension `dim` at the midpoint.
* `_log_soft_volume`: computes the logarithm of the volume of a box using `nn.functional.softplus`.
* `_log_soft_volume_adjusted`: computes the logarithm of the volume of a box using `nn.functional.softplus` and adding a factor of `- 2 * euler_gamma * gumbel_beta`.
* `get_wW`: returns the tuple `z, Z`.

**Methods**

* `_intersection`: computes the intersection between 2 `BoxTensor`. It returns `z` and `Z` of the intersection. Note that it can return flipped boxes, i.e. where `z[i] > Z[i]`.
* `gumbel_intersection_log_volume`: computes a bayesian intersection between 2 `BoxTensor` by calling the `_intersection` method and passing the parameters `bayesian=True` and `gumbel_beta=intersection_temp`. It then computes the logarithm of the volume of the intersection by calling `_log_soft_volume_adjusted` and returns it.
* `intersection`: computes the intersection between 2 BoxTensor by calling the `_intersection` method. It returns the intersection as a `BoxTensor`. Note that it can return flipped boxes, i.e. where `z[i] > Z[i]`.
* `join`: returns the join between 2 `BoxTensor`, as a `BoxTensor`.
* `log_soft_volume`: calls `_log_soft_volume` and returns the result.
* `intersection_log_soft_volume`: computes the intersection between 2 `BoxTensor` by calling the `_intersection` method. It then computes the logarithm of the volume of the intersection by calling `_log_soft_volume` and returns it.


In [ ]:
# box/box_wrapper.py

def _box_shape_ok(t: torch.Tensor, learnt_temp=False) -> bool:
  '''
  Performs the following checks:
  - len(t.shape) should be >= 2
  - if learnt_temp==True -> t.size(-2) should be 4
  - if learnt_temp==False -> t.size(-2) should be 2
  '''
  if len(t.shape) < 2:
      return False
  if not learnt_temp:
      if t.size(-2) != 2:
          return False
      return True
  else:
      if t.size(-2) != 4:
          return False

      return True


def _shape_error_str(tensor_name, expected_shape, actual_shape):
    return "Shape of {} has to be {} but is {}".format(
        tensor_name, expected_shape, tuple(actual_shape)
    )


# see: https://realpython.com/python-type-checking/#type-hints-for-methods
# to know why we need to use TypeVar
TBoxTensor = TypeVar("TBoxTensor", bound="BoxTensor")


class BoxTensor(object):
    """A wrapper which contains a single tensor that
    represents single or multiple boxes.

    Have to use composition instead of inheritance since
    it is not safe to interit from :class:`torch.Tensor`, because
    creating an instance of such a class will always make it a leaf node.
    This works for :class:`torch.nn.Parameter` but won't work for a general
    box_tensor.
    """

    def __init__(self, data: torch.Tensor, learnt_temp: bool = False) -> None:
        """
        .. todo:: Validate the values of z, Z ? z < Z

        Arguments:
            data: Tensor of shape (**, zZ, num_dims). Here, zZ=2, where
                the 0th dim is for bottom left corner and 1st dim is for
                top right corner of the box
        """

        if _box_shape_ok(data, learnt_temp):
            self.data = data
        else:
            raise ValueError(_shape_error_str("data", "(**,2,num_dims)", data.shape))
        super().__init__()

    def __repr__(self):
        return "box_tensor_wrapper(" + self.data.__repr__() + ")"

    @property
    def z(self) -> torch.Tensor:
        """Lower left coordinate as Tensor"""

        return self.data[..., 0, :]

    @property
    def Z(self) -> torch.Tensor:
        """Top right coordinate as Tensor"""

        return self.data[..., 1, :]

    @property
    def box_type(self):
        return "BoxTensor"

    @property
    def centre(self) -> torch.Tensor:
        """Centre coordinate as Tensor"""

        return (self.z + self.Z) / 2

    @classmethod
    def from_zZ(cls: Type[TBoxTensor], z: torch.Tensor, Z: torch.Tensor) -> TBoxTensor:
        """
        Creates a box by stacking z and Z along -2 dim.
        That is if z.shape == Z.shape == (**, num_dim),
        then the result would be box of shape (**, 2, num_dim)
        """

        if z.shape != Z.shape:
            raise ValueError(
                "Shape of z and Z should be the same but is {} and {}".format(
                    z.shape, Z.shape
                )
            )
        box_val: torch.Tensor = torch.stack((z, Z), -2)

        return cls(box_val)

    @classmethod
    def from_split(cls: Type[TBoxTensor], t: torch.Tensor, dim: int = -1) -> TBoxTensor:
        """Creates a BoxTensor by splitting on the dimension dim at midpoint

        Args:
            t: input
            dim: dimension to split on

        Returns:
            BoxTensor: output BoxTensor

        Raises:
            ValueError: `dim` has to be even
        """
        len_dim = t.size(dim)

        if len_dim % 2 != 0:
            raise ValueError(
                "dim has to be even to split on it but is {}".format(t.size(dim))
            )
        split_point = int(len_dim / 2)
        z = t.index_select(
            dim,
            torch.tensor(list(range(split_point)), dtype=torch.int64, device=t.device),
        )

        Z = t.index_select(
            dim,
            torch.tensor(
                list(range(split_point, len_dim)), dtype=torch.int64, device=t.device
            ),
        )

        return cls.from_zZ(z, Z)

    def _intersection(
        self: TBoxTensor,
        other: TBoxTensor,
        gumbel_beta: float = 1.0,
        bayesian: bool = False,
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        t1 = self
        t2 = other
        z, Z = None, None
        if bayesian:
            try:
                z = gumbel_beta * torch.logaddexp(
                    t1.z / gumbel_beta, t2.z / gumbel_beta
                )
                z = torch.max(z, torch.max(t1.z, t2.z))
                Z = -gumbel_beta * torch.logaddexp(
                    -t1.Z / gumbel_beta, -t2.Z / gumbel_beta
                )
                Z = torch.min(Z, torch.min(t1.Z, t2.Z))
            except Exception as e:
                print("Gumbel intersection is not possible")
                # breakpoint()
        else:
            z = torch.max(t1.z, t2.z)
            Z = torch.min(t1.Z, t2.Z)

        return z, Z

    def gumbel_intersection_log_volume(
        self: TBoxTensor,
        other: TBoxTensor,
        volume_temp=1.0,
        intersection_temp: float = 1.0,
        scale=1.0,
    ) -> TBoxTensor:
        z, Z = self._intersection(other, gumbel_beta=intersection_temp, bayesian=True)
        vol = self._log_soft_volume_adjusted(
            z, Z, temp=volume_temp, gumbel_beta=intersection_temp, scale=scale
        )
        return vol

    def intersection(self: TBoxTensor, other: TBoxTensor) -> TBoxTensor:
        """Gives intersection of self and other.

        .. note:: This function can give flipped boxes, i.e. where z[i] > Z[i]
        """
        z, Z = self._intersection(other)

        return self.from_zZ(z, Z)

    def join(self: TBoxTensor, other: TBoxTensor) -> TBoxTensor:
        """Gives join"""
        z = torch.min(self.z, other.z)
        Z = torch.max(self.Z, other.Z)

        return self.from_zZ(z, Z)

    @classmethod
    def _log_soft_volume(
        cls, z: torch.Tensor, Z: torch.Tensor, temp: float = 1.0, scale: Union[float, torch.Tensor] = 1.0
    ) -> torch.Tensor:
        eps = torch.finfo(z.dtype).tiny  # type: ignore

        if isinstance(scale, float):
            s = torch.tensor(scale)
        else:
            s = scale

        return torch.sum(
            torch.log(nn.functional.softplus(Z - z, beta=temp) + 1e-23), dim=-1
        ) + torch.log(
            s
        )  # need this eps so that the derivative of log does not blow

    def log_soft_volume(
        self, temp: float = 1.0, scale: Union[float, torch.Tensor] = 1.0
    ) -> torch.Tensor:
        res = self._log_soft_volume(self.z, self.Z, temp=temp, scale=scale)

        return res

    @classmethod
    def _log_soft_volume_adjusted(
        cls,
        z: torch.Tensor,
        Z: torch.Tensor,
        temp: float = 1.0,
        gumbel_beta: float = 1.0,
        scale: Union[float, torch.Tensor] = 1.0,
    ) -> torch.Tensor:
        eps = torch.finfo(z.dtype).tiny  # type: ignore

        if isinstance(scale, float):
            s = torch.tensor(scale)
        else:
            s = scale

        return (
            torch.sum(
                torch.log(
                    nn.functional.softplus(Z - z - 2 * euler_gamma * gumbel_beta, beta=temp) + 1e-23
                ),
                dim=-1,
            )
            + torch.log(s)
        )

    def intersection_log_soft_volume(
        self,
        other: TBoxTensor,
        temp: float = 1.0,
        gumbel_beta: float = 1.0,
        bayesian: bool = False,
        scale: Union[float, torch.Tensor] = 1.0,
    ) -> torch.Tensor:
        z, Z = self._intersection(other, gumbel_beta, bayesian)
        vol = self._log_soft_volume(z, Z, temp=temp, scale=scale)

        return vol

    @classmethod
    def get_wW(cls, z, Z):
        return z, Z


### DeltaBoxTensor

The `DeltaBoxTensor` class inherits from the `BoxTensor` class. As `BoxTensor`, it is a wrapper for a `torch.Tensor` and represents single or multiple boxes, but it uses a <u>different parametrization</u>.

`DeltaBoxTensor` overrides the following methods and properties.

**Properties**

* `z: torch.Tensor` : lower left coordinates as Tensor.
* `Z: torch.Tensor` : top right coordinates as Tensor.

**Class methods**

* `from_zZ(z, Z)`:
  1. checks if `z` and `Z` have the same shape. If not, it throws an error.
  2. calls `get_wW` to get `w` and `W`.
  3. creates a `torch.Tensor` by concatenating `w` and `W`.
  4. wraps the torch tensor with a `DeltaBoxTensor` and returns it.
* `get_wW(z, Z)`:
  1. checks if `z` and `Z` have the same shape. If not, it throws an error.
  2. calls `_softplus_inverse` to compute `W`.
  3. returns `w` and `W`.
* `from_split(t, dim)`: creates a `DeltaBoxTensor` by splitting the `torch.Tensor` `t `on the dimension `dim` at the midpoint.



In [ ]:
# box/box_wrapper.py


def _softplus_inverse(t: torch.Tensor, beta=1.0, threshold=20):
    below_thresh = beta * t < threshold
    res = t
    res[below_thresh] = torch.log(torch.exp(beta * t[below_thresh]) - 1.0) / beta

    return res


class DeltaBoxTensor(BoxTensor):
    """Same as BoxTensor but with a different parameterization: (**,wW, num_dims)

    z = w
    Z = z + delta(which is always positive)
    """

    @property
    def z(self) -> torch.Tensor:
        return self.data[..., 0, :]

    @property
    def Z(self) -> torch.Tensor:
        z = self.z
        Z = z + nn.functional.softplus(self.data[..., 1, :], beta=10)

        return Z

    @classmethod
    def from_zZ(cls: Type[TBoxTensor], z: Tensor, Z: Tensor) -> TBoxTensor:

        if z.shape != Z.shape:
            raise ValueError(
                "Shape of z and Z should be the same but is {} and {}".format(
                    z.shape, Z.shape
                )
            )
        w, W = cls.get_wW(z, Z)  # type:ignore

        box_val: torch.Tensor = torch.stack((w, W), -2)

        return cls(box_val)

    @classmethod
    def get_wW(cls, z, Z):
        if z.shape != Z.shape:
            raise ValueError(
                "Shape of z and Z should be the same but is {} and {}".format(
                    z.shape, Z.shape
                )
            )
        w = z
        W = _softplus_inverse(Z - z, beta=10.0)  # type:ignore

        return w, W

    @classmethod
    def from_split(cls: Type[TBoxTensor], t: torch.Tensor, dim: int = -1) -> TBoxTensor:
        """Creates a BoxTensor by splitting on the dimension dim at midpoint

        Args:
            t: input
            dim: dimension to split on

        Returns:
            BoxTensor: output BoxTensor

        Raises:
            ValueError: `dim` has to be even
        """
        len_dim = t.size(dim)

        if len_dim % 2 != 0:
            raise ValueError(
                "dim has to be even to split on it but is {}".format(t.size(dim))
            )
        split_point = int(len_dim / 2)
        w = t.index_select(
            dim,
            torch.tensor(list(range(split_point)), dtype=torch.int64, device=t.device),
        )

        W = t.index_select(
            dim,
            torch.tensor(
                list(range(split_point, len_dim)), dtype=torch.int64, device=t.device
            ),
        )
        box_val: torch.Tensor = torch.stack((w, W), -2)

        return cls(box_val)

### BoxEmbedding

The `BoxEmbedding` class inherits from `torch.nn.Embedding`, a lookup table that maps an index value to a weight matrix of a certain dimension. It constitutes the model's embedding layer and it is similar to [AllenNLP](https://github.com/allenai/allennlp/tree/main) embedding, but it returns box tensors by splitting the output of usual embeddings into `z` and `Z`.

The **constructor** takes in input the following parameters and initializes the layer's weights:
 * `num_embeddings: int`: the size of the dictionary of embeddings, i.e. the number of embeddings.
 * `box_embedding_dim: int`: the dimension of each embedding. The number of weights will be `box_embedding_dim*2`.
 * `box_type`: the type of the box. By default it is set to `"BoxTensor"`.
 * `init_interval_center`: By default it is set to `0.25`.
 * `init_interval_delta`: By default it is set to `0.1`.


`BoxEmbedding` exposes the following methods and properties.

**Properties**

* `all_boxes: TBoxTensor` : returns all the box embeddings.

**Methods**

* `init_weights`: initializes the layer's weights from a uniform distribution.
* `forward`:
  - calls the `nn.Embedding.forward` method;
  - call the `from_split` method of `DeltaBoxTensor` or `BoxTensor`, depending on the value of the `box_type` paramer passed to the `BoxEmbedding` constructor.
* `get_volumes`: returns the `log_soft_volume` of the box embeddings.

In [ ]:
# box/modules.py

def _uniform_small(weight, emb_dim, param1, param2, box_type):
    with torch.no_grad():
        temp = torch.zeros_like(weight)
        torch.nn.init.uniform_(temp, 0.0 + 1e-7, 1.0 - 0.1 - 1e-7)
        # z = torch.min(temp[..., :emb_dim], temp[..., emb_dim:])
        z = temp[..., :emb_dim]
        Z = z + 0.1
        w, W = box_type.get_wW(z, Z)
        weight[..., :emb_dim] = w
        weight[..., emb_dim : emb_dim * 2] = W


class BoxEmbedding(nn.Embedding):
    box_types = {
        "DeltaBoxTensor": DeltaBoxTensor,
        "BoxTensor": BoxTensor,
    }

    def init_weights(self):
        _uniform_small(
            self.weight,
            self.box_embedding_dim,
            0.0 + 1e-7,
            1.0 - 1e-7,
            self.box_types[self.box_type],
        )

    def __init__(
        self,
        num_embeddings: int,
        box_embedding_dim: int,
        box_type="BoxTensor",
        init_interval_center=0.25,
        init_interval_delta=0.1,
    ) -> None:
        """Similar to allennlp embeddings but it returns box
        tensor by splitting the output of usual embeddings
        into z and Z

        Arguments:
            box_embedding_dim: Embedding weight would be box_embedding_dim*2
        """
        vector_emb_dim = box_embedding_dim * 2
        if box_type == "BoxTensorLearntTemp":
          # it will never enter here because box_type can only be "BoxTensor" or
          # "DeltaBoxTensor"
            vector_emb_dim = box_embedding_dim * 4

        super().__init__(num_embeddings, vector_emb_dim)
        self.box_type = box_type
        self.init_interval_delta = init_interval_delta
        self.init_interval_center = init_interval_center
        try:
            self.box = self.box_types[box_type]
        except KeyError as ke:
            raise ValueError("Invalid box type {}".format(box_type)) from ke
        self.box_embedding_dim = box_embedding_dim
        self.init_weights()

    def forward(self, inputs: torch.LongTensor):
        emb = super().forward(inputs)  # shape (**, self.box_embedding_dim*2)
        box_emb = self.box.from_split(emb)
        return box_emb

    def get_volumes(self, temp: Union[float, torch.Tensor]) -> torch.Tensor:
        return self.all_boxes.log_soft_volume(temp=temp)

    @property
    def all_boxes(self) -> TBoxTensor:
        all_index = torch.arange(
            0, self.num_embeddings, dtype=torch.long, device=self.weight.device
        )
        all_ = self.forward(all_index)

        return all_


## 3.3 models

### BaseModule

The `BaseModule` class inherits from `torch.nn.Module` and exposes the methods needed to save and load the model's parameters and the training checkpoints.

To do so, it uses the following methods defined in `torch.nn.Module`:

* `load_state_dict`: https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.load_state_dict

* `eval`
* `state_dict`

In [ ]:
# models/BaseModule.py

class BaseModule(nn.Module):
    def __init__(self):
        super(BaseModule, self).__init__()
        self.zero_const = nn.Parameter(torch.Tensor([0]))
        self.zero_const.requires_grad = False
        self.pi_const = nn.Parameter(torch.Tensor([3.14159265358979323846]))
        self.pi_const.requires_grad = False

    def load_checkpoint(self, path):
        self.load_state_dict(torch.load(os.path.join(path), map_location="cpu"))
        self.eval()

    def save_checkpoint(self, path):
        torch.save(self.state_dict(), path)

    def load_parameters(self, path):
        f = open(path, "r")
        parameters = json.loads(f.read())
        f.close()
        for i in parameters:
            parameters[i] = torch.Tensor(parameters[i])
        self.load_state_dict(parameters, strict=False)
        self.eval()

    def save_parameters(self, path):
        f = open(path, "w")
        f.write(json.dumps(self.get_parameters("list")))
        f.close()

    def get_parameters(self, mode="numpy", param_dict=None):
        all_param_dict = self.state_dict()
        if param_dict == None:
            param_dict = all_param_dict.keys()
        res = {}
        for param in param_dict:
            if mode == "numpy":
                res[param] = all_param_dict[param].cpu().numpy()
            elif mode == "list":
                res[param] = all_param_dict[param].cpu().numpy().tolist()
            else:
                res[param] = all_param_dict[param]
        return res

    def set_parameters(self, parameters):
        for i in parameters:
            parameters[i] = torch.Tensor(parameters[i])
        self.load_state_dict(parameters, strict=False)
        self.eval()


### Word2Box

The `Word2Box` class inherits from `BaseModule`.

The **constructor** takes in input the following parameters:
* `TEXT`: an object that contains some text statistics, used to get the size of the vocabulary.
* `embedding_dim`: the dimension of the `BoxEmbedding` layers. By default it is set to `50`.
* `batch_size`: the batch size. By default it is set to `10`.
* `n_gram`: the window size. By default it is set to `4`.
* `volume_temp`: volume temperature to adjust the "sharpness" of the distribution. A high temperature makes the distribution more uniform, while a low temperature makes it more sharply peaked. By default it is set to `1.0`.
* `intersection_temp`: the beta coeffiencient of the Gumbel distribution. By default it is set to `1.0`.
* `box_type`: the type of the box. By default it is set to `"BoxTensor"`.

It initializes the model and creates the `BoxEmbedding` layers for the target and context words.

`Word2Box` defines the following methods:

* `forward`:
  - applies the `BoxEmbedding` layers to the context and target words;
  - unsqueezes the training data;
  - computes the score choosing the method based on the value of the `intersection_temp` parameter.
* `word_similarity(w1, w2)`: computes the word embedding for each of the input words `w1` and `w2`, and then it computes the volume of the intersection between the 2 box embeddings. The method used to compute the volume of the intersection is chosen based on the value of the `intersection_temp` parameter.
* `conditional_similarity(w1, w2)`: same as `word_similary`, plus it computes the `_log_soft_volume_adjusted`.

In [ ]:
# models/word2box.py

class Word2Box(BaseModule):
    def __init__(
        self,
        TEXT=None,
        embedding_dim=50,
        batch_size=10,
        n_gram=4,
        volume_temp=1.0,
        intersection_temp=1.0,
        box_type="BoxTensor",
        **kwargs
    ):
        super(Word2Box, self).__init__()

        # Model
        self.batch_size = batch_size
        self.n_gram = n_gram
        self.vocab_size = len(TEXT.itos)
        self.embedding_dim = embedding_dim

        # Box features
        self.volume_temp = volume_temp
        self.intersection_temp = intersection_temp
        self.box_type = box_type

        # Create embeddings
        self.embeddings_word = BoxEmbedding(
            self.vocab_size, self.embedding_dim, box_type=box_type
        )
        self.embedding_context = BoxEmbedding(
            self.vocab_size, self.embedding_dim, box_type=box_type
        )

    def forward(self, idx_word, idx_context, context_mask, train=True):
        context_boxes = self.embedding_context(idx_context)  # Batch_size * 2 * dim
        word_boxes = self.embeddings_word(idx_word)  # Batch_size * ns+1 * 2 * dim
        if train == True:
            word_boxes.data.unsqueeze_(
                1
            )  # Braodcast the word vector to the the context + negative_samples.

        if self.intersection_temp == 0.0:
            score = word_boxes.intersection_log_soft_volume(
                context_boxes, temp=self.volume_temp
            )
        else:
            score = word_boxes.gumbel_intersection_log_volume(
                context_boxes,
                volume_temp=self.volume_temp,
                intersection_temp=self.intersection_temp,
            )

        return score

    def word_similarity(self, w1, w2):
        with torch.no_grad():
            word1 = self.embeddings_word(w1)
            word2 = self.embeddings_word(w2)
            if self.intersection_temp == 0.0:
                score = word1.intersection_log_soft_volume(word2, temp=self.volume_temp)
            else:
                score = word1.gumbel_intersection_log_volume(
                    word2,
                    volume_temp=self.volume_temp,
                    intersection_temp=self.intersection_temp,
                )
            return score

    def conditional_similarity(self, w1, w2):
        with torch.no_grad():
            word1 = self.embeddings_word(w1)
            word2 = self.embeddings_word(w2)
            if self.intersection_temp == 0.0:
                score = word1.intersection_log_soft_volume(word2, temp=self.volume_temp)
            else:
                score = word1.gumbel_intersection_log_volume(
                    word2,
                    volume_temp=self.volume_temp,
                    intersection_temp=self.intersection_temp,
                )

            # Word1 Word2  queen   royalty 5.93
            # Word2 is more general P(royalty | queen) = 1
            # Thus we need p(w2 | w1)
            score -= word1._log_soft_volume_adjusted(
                word1.z,
                word1.Z,
                temp=self.volume_temp,
                gumbel_beta=self.intersection_temp,
            )

            return score


### Word2BoxConjunction

`Word2BoxConjunction` is the CBOW implementation of the Word2Box algorithm. It inherits from the `Word2Box` class and overrides the `forward` method of `Word2Box`. Furthermore,
it defines a new method called `intersect_multiple_box`, which applies a mask to the input boxes and turns them into Gumbel random variables.

* `forward`: similar to `Word2Box`'s `forward` method, but instead of using the box embeddings of the context as is, it applies to it a further trasformation defined by the `intersect_multiple_box` method.
* `intersect_multiple_box`:
    - Applies a mask to the boxes.
    - Then, it applies a transformation to `z` and `Z` in order to make them Gumbel random variables.
    - Finally, it returns a new `BoxTensor` with the resulting `z` and `Z`.
    - Note that <u>`intersection_temp` should always be `!=0`</u>, otherwise there would be a division by 0.


In [ ]:
# models/word2box.py

class Word2BoxConjunction(Word2Box):

    def intersect_multiple_box(self, boxes, mask):
        beta = self.intersection_temp
        z = boxes.z.clone()
        Z = boxes.Z.clone()

        z[~mask] = float("-inf")
        Z[~mask] = float("inf")
        z = beta * torch.logsumexp(z / beta, dim=1, keepdim=True)
        Z = -beta * torch.logsumexp(-Z / beta, dim=1, keepdim=True)

        return BoxTensor.from_zZ(z, Z)

    def forward(self, idx_word, idx_context, mask_context, train=True):
        word_boxes = self.embeddings_word(idx_word)  # Batch_size * ns+1 * 2 * dim
        context_boxes = self.embedding_context(idx_context)  # Batch_size * 2 * dim

        # Note that the context is not masked yet. We need to mask them as well.
        pooled_context = self.intersect_multiple_box(context_boxes, mask_context)

        if self.intersection_temp == 0.0:
          # it will never enter here
            score = word_boxes.intersection_log_soft_volume(
                pooled_context, temp=self.volume_temp
            )
        else:
            score = word_boxes.gumbel_intersection_log_volume(
                pooled_context,
                volume_temp=self.volume_temp,
                intersection_temp=self.intersection_temp,
            )
        return score

## 3.4 train

### Loss functions

* **Noise Contrastive Estimation** (`nce`) loss function:
  + can be used for any embeddings;
  + *Word2Vec* uses this loss function;
  + we pass the *unnormalised* probabilities through sigmoid to normalise the score.

  + input:
	    pos: Unnormalised similarity score for positives.
	    neg: Unnormalised similarity score for negatives
  + output:
	    loss = - (logsigmoid(pos) + sum(logsigmoid(-neg)))

* **Negative Log Likelihood** (`nll`) loss for box embeddings:
  + input:
	    pos = log probabiltiy for positive examples
	    neg = log probabiltiy for negaitve examples
  + output:
	    loss = - (pos + sum(log(1-exp(neg)))

* **Max Margin** (`max_margin`) loss for box embeddings:
  + used to encourage the model to correctly rank the correct output higher than any of the incorrect outputs. This is achieved by maximizing the margin between the score of the correct output and the scores of the incorrect outputs. The max-margin loss helps the model learn to make more accurate predictions by emphasizing the importance of the correct output being clearly distinguished from the incorrect ones.
  + input scores can be un-normalised;
  + goal: increase the pos similarity score more than a margin from the negative scores. If that margin is satisfied then the loss is zero.

  + input:
	    pos: Unnormalised similarity (may be log in case of Boxes) score for positives.
	    neg: Unnormalised similarity (may be log in case of Boxes) score for negatives.
  + output:
	    loss =  - max(0, pos - mean(neg) + margin)

In [ ]:
# train/loss.py

def nll(pos, neg, **kwagrs):
    """
	The Negative Log Likelihood loss funtion is used for
    box embeddings.

	Args:
	    pos = log probabiltiy for positive examples
	    neg = log probabiltiy for negaitve examples
	Output:
	    loss = - (pos + sum(log(1-exp(neg)))
	"""
    assert (pos < 0).all(), "Log probabiltiy can not be positive"
    assert (neg < 0).all(), "Log probabiltiy can not be positive"
    return -(pos + torch.sum(log1mexp(neg), dim=1)) # log1mexp is defined in box.utils


def nce(pos, neg, **kwagrs):
    """
	The Noise Contrastive Estimation loss function can be
    used for any embeddings.
	However, here we pass the unnormalised probabilities
	through sigmoid to normalised the score. Word2vec uses
	this loss function.

	Args:
	    pos: Unnormalised similarity score for positives.
	    neg: Unnormalised similarity score for negatives.
	Output:
	    loss = -(logsigmoid(pos) + sum(logsigmoid(-neg)))
	"""
    return -(nn.functional.logsigmoid(pos) + torch.sum(nn.functional.logsigmoid(-neg), dim=1))


def max_margin(pos, neg, margin=5.0):
    """
	This is max margin loss for box embeddings.
	Here, the input scores can be un-normalised. The object here
	is to make increse the pos similarity score more than a margin
	from the negative scores. If that margin is satisfied then the
	loss is zero.

	Args:
	    pos: Unnormalised similarity(maybe log in case of Boxes) score for positives.
	    neg: Unnormalised similarity(maybe log in case of Boxes) score for negatives.
	Output:
	    loss =  - max(0, pos - mean(neg) + margin)
	"""
    # Replicate the positive score number of negative sample times
    zero = torch.tensor(0.0).to(device)
    return torch.sum(torch.max(zero, neg - pos + margin), dim=1)


### Negative Sampling

**RandomNegativeCBOW**

The class `RandomNegativeCBOW` is used to perform the negative sampling for the models based on *CBOW*.

The **constructor** takes in input the following parameters:
* `number_of_samples: int`: number of negative samples for each positive sample. By default it is set to `5`.
* `sampling_distn: torch.LongTensor`: the sampling distribution; it's a tensor containing probabilities.

`RandomNegativeCBOW` implements the `__call__(batch)` method, which samples the negative examples by using the `torch.multinomial` method. The negative samples are appended to the `batch`'s `center_word` and then this new batch is returned.


**RandomNegativeSkipGram**

The class `RandomNegativeSkipGram` inherits from `RandomNegativeCBOW` and it is used to perform the negative sampling for the models based on *SkipGram*.

It overrides the `__call__(batch)` method. This method appends the negative samples to the `batch`'s `context_words`.

In [ ]:
# train/negative_sampling.py

class RandomNegativeCBOW:
    """
    This augments a batch of data to include randomly sampled target center words.
    Appends the sampled words with 'center_words' of the batch
    """

    def __init__(self, number_of_samples: int = 5, sampling_distn: torch.LongTensor = None):
        self.number_of_samples = number_of_samples
        self.sampling_distn = sampling_distn

    def __call__(self, batch) -> torch.LongTensor:
        x, y = batch["context_words"].shape
        negatives = torch.multinomial(
            self.sampling_distn,
            num_samples=self.number_of_samples * x,
            replacement=True,
        ).resize(x, self.number_of_samples)
        batch["center_word"] = torch.cat(
            (batch["center_word"].unsqueeze(1), negatives), dim=-1
        )
        return batch


class RandomNegativeSkipGram(RandomNegativeCBOW):
    """
    This augments a batch of data to include randomly sampled target context words.
    Appends the sampled words with 'context_words' of the batch
    """

    def __call__(self, batch) -> torch.LongTensor:
        x, y = batch["context_words"].shape
        negatives = torch.multinomial(
            self.sampling_distn,
            num_samples=self.number_of_samples * x * y,
            replacement=True,
        ).resize(x, y, self.number_of_samples)
        batch["context_words"] = torch.cat(
            (batch["context_words"].unsqueeze(-1), negatives), dim=-1
        )
        return batch


### Trainer

The `Trainer` class exposes the methods needed to train the models defined in the `models` folder.

The **constructor** takes in input the following parameters:
* `train_iter: LazyDatasetLoader`: an instance of `LazyDatasetLoader` for the training dataset.
* `val_iter: LazyDatasetLoader`: an instance of `LazyDatasetLoader` for the validation dataset.
* `vocab`: an instance of `torchtext.data.Field()`. This object has two main attributes: `stoi` (string-to-index) and `itos` (index-to-string).
`vocab.stoi` is a dictionary that maps each unique word in the vocabulary to a unique index. This is useful for converting a sequence of words into a sequence of indices, which can then be input to a model. `vocab.itos` is a list where the element at each index is the corresponding word. This is useful for converting a sequence of indices back into a sequence of words.
* `lr: float`: the learning rate. By default it is set to `0.001`.
* `n_gram: int`: the window size. By default it is set to `4`.
* `loss_fn: str`: the loss function. It can be `'nce'`, `'nll'` or `'max-margin'`.
* `negative_samples: int`: number of negative samples for training. By default it is set to `5`.
* `log_frequency: int`: the log frequency.  By default it is set to `1000`.

It also sets the frequency of `'<pad>'` to `0`, to avoid to sample it during the negative sampling.


**Methods**

* `train_model`: Not implemented.


In [ ]:
# train/Trainer.py

torch.autograd.set_detect_anomaly(True)

criterions = {"nll": nll, "nce": nce, "max_margin": max_margin}


class Trainer:
    def __init__(
        self,
        train_iter,
        val_iter,
        vocab,
        lr=0.001,
        n_gram=4,
        loss_fn=None,
        negative_samples=5,
        log_frequency=1000,
    ):
        self.train_iter = train_iter
        self.val_iter = val_iter
        self.n_gram = n_gram
        self.lr = lr
        self.vocab = vocab
        self.vocab_size = len(self.vocab.itos)
        self.negative_samples = negative_samples
        self.log_frequency = log_frequency
        self.vocab.freqs["<pad>"] = 0  # Don't want to negative sample pads.
        sorted_freqs = torch.tensor(
            [self.vocab.freqs.get(key, 0) for key in self.vocab.itos]
        )
        self.sampling = torch.pow(sorted_freqs, 0.75)
        self.sampling = self.sampling / torch.sum(self.sampling)
        if use_cuda:
            self.sampling = self.sampling.cuda()

    def train_model(self, model, num_epochs=100, path="./", save_model=False):
        pass


### TrainerWordSimilarity

The `TrainerWordSimilarity` class inherits from `Trainer`.

The **constructor** takes in input the following parameters:

* `train_iter: LazyDatasetLoader`: an instance of `LazyDatasetLoader` for the training dataset.
* `val_iter: LazyDatasetLoader`: an instance of `LazyDatasetLoader` for the validation dataset.
* `vocab`: an instance of `torchtext.data.Field()`. This object has two main attributes: `stoi` (string-to-index) and `itos` (index-to-string).
`vocab.stoi` is a dictionary that maps each unique word in the vocabulary to a unique index. This is useful for converting a sequence of words into a sequence of indices, which can then be input to a model. `vocab.itos` is a list where the element at each index is the corresponding word. This is useful for converting a sequence of indices back into a sequence of words.
* `lr: float`: the learning rate. By default it is set to `0.001`.
* `n_gram: int`: the window size. By default is set to `4`.
* `loss_fn: str`: the loss function. It can be `'nce'`, `'nll'` or `'max-margin'`. By default it is set to `'max_margin'`
* `negative_samples: int`: number of negative samples for training. By default it is set to `5`.
* `log_frequency: int`: the log frequency.  By default it is set to `1000`.
* `model_mode: str`: indicates whether the model is based on *SkipGram* or *CBOW*. It can be `'SkipGram'` or `'CBOW'`. By default it is set to `'CBOW'`.
* `margin: float`: the margin applicable for `max_margin` loss. By default is set to `0.0`.
* `similarity_datasets_dir`: path to the similarity datasets dir.
* `subsampling_prob`: it's not used since the `training` method passes a `None` value.

It performs the negative sampling calling `RandomNegativeCBOW` or `RandomNegativeSkipGram`, depending on the value of `model_mode`.

**Methods**

* `train_model(model, num_epochs, path, save_model)`: this method is used to train a given model for a specified number of epochs. It uses the Adam optimizer and a custom loss function. The method also includes a training loop that iterates over the training data, computes the loss, and updates the model parameters. It also checks for NaN or infinite values in the loss or parameters, which could indicate problems with training. The method also includes code for intermediate and final evaluation of the model. It computes a metric (spearman correlation) on the **Simlex-999** dataset and keeps track of the best value of this metric. If the `save_model` flag is set, it saves the current model and the best model (according to the metric) to disk.

* `model_eval(model)`:
  - If the `similarity_datasets_dir` is not `None`, it evaluates the model using `model.word_similarity` on each row of each dataset present in the folder.
  - Then, it computes the `spearmanr` correlation between the predicted scores and the target scores.
  - It returns a dictionary containing the correlation value obtained for each similarity dataset.
* `to(batch, device)`: calls the `.to(device)` method for each item in the batch.


In [ ]:
# train/Trainer.py

class TrainerWordSimilarity(Trainer):
    """TrainerWordSimilarity"""

    def __init__(
        self,
        train_iter,
        val_iter,
        vocab,
        lr=0.001,
        n_gram=4,
        loss_fn="max_margin",
        negative_samples=5,
        model_mode="CBOW",
        log_frequency=1000,
        margin=0.0,
        similarity_datasets_dir=None,
        subsampling_prob=None,
    ):
        super(TrainerWordSimilarity, self).__init__(
            train_iter,
            val_iter,
            vocab,
            lr=lr,
            n_gram=n_gram,
            loss_fn=loss_fn,
            negative_samples=negative_samples,
            log_frequency=log_frequency,
        )

        self.similarity_datasets_dir = similarity_datasets_dir
        self.margin = margin
        self.loss_fn = criterions[loss_fn]
        # If subsampling has been done earlier, then word count must have been changed
        # This is an expected word count based on the subsampling prob parameters.
        if subsampling_prob != None:
          # it never enters here because the value of subsampling_prob is always == None
            self.sampling = (
                torch.min(torch.tensor(1.0).to(device), 1 - subsampling_prob.to(device))
                * self.sampling
            )
        if model_mode == "CBOW":
            self.add_negatives = RandomNegativeCBOW(negative_samples, self.sampling)
        elif model_mode == "SkipGram":
            self.add_negatives = RandomNegativeSkipGram(negative_samples, self.sampling)

    def to(self, batch, device):
        for k, v in batch.items():
            batch[k] = batch[k].to(device)
        return batch

    def train_model(
        self, model, num_epochs=100, path="./checkpoints", save_model=False
    ):
        ## Setting up the optimizers
        parameters = filter(lambda p: p.requires_grad, model.parameters())
        optimizer = torch.optim.Adam(params=parameters, lr=self.lr)
        metric = {}
        best_simlex_ws = -1
        ## Setting Up the loss function
        for epoch in tqdm(range(num_epochs)):
            epoch_loss = []
            model.train()

            for i, batch in enumerate(tqdm(self.train_iter)):
                # Create negative samples for the batch
                batch = self.to(batch, device)
                batch = self.add_negatives(batch)

                # Start the optimization
                optimizer.zero_grad()
                score = model.forward(
                    batch["center_word"],
                    batch["context_words"],
                    batch["context_mask"],
                    train=True,
                )
                assert (
                    score.shape[-1] == self.negative_samples + 1
                )  # check the shape of the score

                # Score log_intersection_volume (un-normalised) for Word2Box
                pos_score = score[..., 0].reshape(
                    -1, 1
                )  # The first element correspond to the Positive
                neg_score = score[..., 1:].reshape(
                    -1, self.negative_samples
                )  # The rest of the elements are for negative samples
                # Calculate Loss
                loss = self.loss_fn(
                    pos_score, neg_score, margin=self.margin
                )  # Margin is not required for nll or nce
                # Handled through kwargs in loss.py
                total_loss = torch.sum(loss)
                avg_loss = torch.mean(loss)
                if torch.isnan(loss).any():
                    raise RuntimeError("Loss value is nan :(")

                # Back-propagation
                total_loss.backward()
                optimizer.step()

                for param in model.parameters():
                    if torch.isinf(param).any():
                        raise RuntimeError("parameters went to infinity")
                    if torch.isnan(param).any():
                        raise RuntimeError("parameters went to nan")
                    if param.grad is not None:
                        if torch.isnan(param.grad).any():
                            raise RuntimeError("Gradient went to nan")

                epoch_loss.append(avg_loss.data.item())

                # Intermediate eval
                if i % int(len(self.train_iter) / self.log_frequency) == 0:
                    # Start model eval
                    model.eval()
                    ws_metric = self.model_eval(
                        model
                    )  # This ws_metric contains correlations
                    metric.update({"epoch_loss": np.mean(epoch_loss)})
                    # Update the metric for wandb login
                    metric.update(ws_metric)

                    simlex_ws = metric["En-Simlex-999.Txt"]
                    best_simlex_ws = max(metric["En-Simlex-999.Txt"], best_simlex_ws)
                    metric.update({"best_simlex_ws": best_simlex_ws})
                    print(
                        "Epoch {0} | Loss: {1}| spearmanr: {2}".format(
                            epoch + 1, np.mean(epoch_loss), simlex_ws
                        )
                    )

                    if save_model:
                        model.save_checkpoint(Path(path) / "model.ckpt")
                        # savethe best hyperparameter
                        if simlex_ws == best_simlex_ws:
                            model.save_checkpoint(Path(path) / "best_model.ckpt")

                    model.train()

            # Logging training loss
            metric.update({"epoch_loss": np.mean(epoch_loss)})

            model.eval()
            ws_metric = self.model_eval(model)  # This ws_metric contains correlations

            # Update the metric
            metric.update(ws_metric)

            simlex_ws = metric["En-Simlex-999.Txt"]
            best_simlex_ws = max(simlex_ws, best_simlex_ws)
            metric.update({"best_simlex_ws": best_simlex_ws})
            print(
                "Epoch {0} | Loss: {1}| spearmanr: {2}".format(
                    epoch + 1, np.mean(epoch_loss), simlex_ws
                )
            )

            if save_model:
                model.save_checkpoint(Path(path) / "model.ckpt")
                # save the best hyperparameter
                if simlex_ws > best_simlex_ws:
                    model.save_checkpoint(Path(path) / "best_model.ckpt")

        print("Model trained.")
        print("Output saved.")

    def model_eval(self, model):
        if self.similarity_datasets_dir == None:
            return 0

        metrics = {}
        correlation = 0.0

        # similarity_file is expected to be in the format word1\tword2\tscore
        if self.similarity_datasets_dir is not None and self.vocab is not None:
            file_list = os.listdir(self.similarity_datasets_dir)
            for file in file_list:
                with xopen(os.path.join(self.similarity_datasets_dir, file)) as f:
                    reader = csv.reader(f, delimiter="\t")
                    real_scores = []
                    predicted_scores = []
                    missing_count = 0
                    total_count = 0
                    for row in reader:
                        row[0] = row[0].lower()
                        row[1] = row[1].lower()
                        if (
                            self.vocab.stoi.get(row[0], "<unk>") != "<unk>"
                            and self.vocab.stoi.get(row[1], "<unk>") != "<unk>"
                        ):
                            word1 = (
                                torch.tensor(self.vocab.stoi[row[0]], dtype=int)
                                .unsqueeze(0)
                                .to(device)
                            )
                            word2 = (
                                torch.tensor(self.vocab.stoi[row[1]], dtype=int)
                                .unsqueeze(0)
                                .to(device)
                            )
                            score = model.word_similarity(word1, word2)
                            if file.title() == "Hyperlex-Dev.Txt":
                                score = model.conditional_similarity(word1, word2)

                            predicted_scores.append(score.item())
                            real_scores.append(float(row[2]))
                        else:
                            missing_count += 1
                        total_count += 1
                    # print(f"{file.title()} missing data point: {missing_count} out of {total_count}")
                    # Calculate spearman's rank correlation coefficient between predicted scores and real scores
                    correlation = spearmanr(predicted_scores, real_scores)[0]
                    metrics[file.title()] = correlation

        pprint.pprint(metrics, width=1)

        return metrics


### Train

The `training` function takes in input the configurations needed to run the training of the model.
- It creates a `LazyDatasetLoader` instance for the training dataset.
- It creates the correct model, based on the `config["model_type"]` value.
- It creates a trainer of type `TrainerWordSimilarity`.
- It calls the method `trainer.train_model` to start the model's training.

I've changed the function to make it return the `model` and the `trainer`, since we will need them later to test the model.

In [ ]:
# train/train.py

def training(config):

    # Set the seed
    if config["seed"] is None:
        config["seed"] = random.randint(0, 2**32)
    torch.manual_seed(config["seed"])
    random.seed(config["seed"])

    # get_iter_on_device is defined in datasets.utils
    TEXT, train_iter = get_iter_on_device(
        config["batch_size"],
        config["dataset"],
        config["n_gram"],
        config["subsample_thresh"],
        config["data_device"],
        config["eos_mask"],
    )

    if config["model_type"] == "Word2Box":
        model = Word2Box(
            TEXT=TEXT,
            embedding_dim=config["embedding_dim"],
            batch_size=config["batch_size"],
            n_gram=config["n_gram"],
            intersection_temp=config["int_temp"],
            volume_temp=config["vol_temp"],
            box_type=config["box_type"],
            pooling=config["pooling"],
        )


    elif config["model_type"] == "Word2BoxConjunction":
        model = Word2BoxConjunction(
            TEXT=TEXT,
            embedding_dim=config["embedding_dim"],
            batch_size=config["batch_size"],
            n_gram=config["n_gram"],
            intersection_temp=config["int_temp"],
            volume_temp=config["vol_temp"],
            box_type=config["box_type"],
        )

    else:
        raise ValueError("Model type is not valid. Please enter a valid model type")

    if use_cuda:
        model.cuda()

    # Instance of trainer
    if config["model_type"] == "Word2Box":
        model_mode = "SkipGram"
    elif config["model_type"] == "Word2BoxConjunction":
        model_mode = "CBOW"

    trainer = TrainerWordSimilarity(
        train_iter=train_iter,
        val_iter=None,
        vocab=TEXT,
        lr=config["lr"],
        n_gram=config["n_gram"],
        loss_fn=config["loss_fn"],
        negative_samples=config["negative_samples"],
        model_mode=model_mode,
        log_frequency=config["log_frequency"],
        margin=config["margin"],
        similarity_datasets_dir=config["eval_file"],
        subsampling_prob=None,  # pass: subsampling_prob, when you want to adjust neg_sampling distn
    )

    trainer.train_model(
        model=model,
        num_epochs=config["num_epochs"],
        path=config.get("save_dir", False),
        save_model=config.get("save_model", False),
    )

    return model, trainer


# 4 - Training

In this section, we are going to train the `Word2BoxConjunction` model. But before doing so, we need to do the following things:
- upload the training dataset on Colab. We can use an extract of the **Enwik8** dataset (https://mattmahoney.net/dc/textdata.html), that I previously preprocessed. Alternatively, we can use the dataset provided by the repository, which is a part of the **Penn Treebank** dataset and it's located in the `word2box/data/ptb` folder.
- create the `config` dictionary, which contains the configuration needed by the training algorithm.
- create the directory where the trained models will be saved.


## 4.1 Upload the dataset on Colab
> If you want to train the model using the **Penn Treebank** dataset, you can skip this section.


Before uploading the dataset, we need to create the folder where we will save the training data. To do so, we need to keep in mind the following requirements:
- the folder should be placed inside the `data` directory;
- the folder name (in this case `enwik8-preprocessed`) should be passed as dataset name in the algorithm configurations.

In [ ]:
! rm -rf /content/word2box/data/enwik8-preprocessed
! mkdir -p /content/word2box/data/enwik8-preprocessed

In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1--8USt54TH2SDL2YzRoBm7EhoRE4EuX9',
                                    dest_path='/content/word2box/data/enwik8-preprocessed/train.txt')

## 4.2 Training parameters and configurations

I initialize most of the configuration parameters using the default values provided by the authors. I use only 1 training epoch due to time and hardware constraints.

In [ ]:
SAVED_MODELS_DIR = '/content/saved-models'

# DATASET_NAME = 'ptb' # Penn Treebank
DATASET_NAME = 'enwik8-preprocessed' # Enwik8 preprocessed

config = {
    "alpha_dim": 32,
    "batch_size": 1024,
    "cuda": True,
    "data_device": 'cpu',
    "diag_context": 0,
    "eval_file": "./data/similarity_datasets/",
    "eos_mask": True,
    "embedding_dim": 50,
    "int_temp": 0.01,
    "log_frequency": 100,
    "lr": 0.01,
    "margin": 10,
    "n_gram": 4,
    "negative_samples": 5,
    "sep_output": 0,
    "subsample_thresh": 0.001,
    "vol_temp": 1.0,

    "num_epochs": 1,
    "box_type": "DeltaBoxTensor",
    "dataset": DATASET_NAME,
    "loss_fn": "nce",
    "model_type": "Word2BoxConjunction",
    "save_dir": SAVED_MODELS_DIR,
    "save_model": True,
    "seed": 5,
}

In [ ]:
# create the save_dir folder
! mkdir /content/saved-models

%cd /content/word2box
model, trainer = training(config)

/content/word2box
Loading VOCAB & Tokenized Training files ...
Creating iterable dataset ...
Creating iterable dataset on GPU/CPU...


  0%|          | 0/7370 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/_tensor.py:893: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


{'En-Mc-30.Txt': -0.015522977529359997,
 'En-Men-Tr-3K.Txt': 0.016517837806777076,
 'En-Mturk-287.Txt': 0.0016186324829713758,
 'En-Mturk-771.Txt': 0.05519493695610718,
 'En-Rg-65.Txt': -0.001639452637944709,
 'En-Rw-Stanford.Txt': 0.033186378737330956,
 'En-Simlex-999.Txt': -0.04636905200715898,
 'En-Simverb-3500.Txt': 0.018994889783990804,
 'En-Verb-143.Txt': 0.03267956472000217,
 'En-Ws-353-All.Txt': -0.04734796087208728,
 'En-Ws-353-Rel.Txt': -0.09566764781278951,
 'En-Ws-353-Sim.Txt': 0.024757073121765068,
 'En-Yp-130.Txt': -0.14308218803755457}
Epoch 1 | Loss: 39.35222244262695| spearmanr: -0.04636905200715898



  1%|          | 75/7370 [00:28<2:20:21,  1.15s/it]

{'En-Mc-30.Txt': 0.08426759230224,
 'En-Men-Tr-3K.Txt': 0.032680797132741565,
 'En-Mturk-287.Txt': 0.1835035184732608,
 'En-Mturk-771.Txt': -0.001891978299496464,
 'En-Rg-65.Txt': -0.05952270787118613,
 'En-Rw-Stanford.Txt': -0.08810699320690556,
 'En-Simlex-999.Txt': -0.0833181369461044,
 'En-Simverb-3500.Txt': -0.008612081631543705,
 'En-Verb-143.Txt': 0.29526614188151523,
 'En-Ws-353-All.Txt': -0.0641912552397264,
 'En-Ws-353-Rel.Txt': -0.002437886494290873,
 'En-Ws-353-Sim.Txt': -0.039888671659203824,
 'En-Yp-130.Txt': 0.04023926245252218}
Epoch 1 | Loss: 28.16942441785658| spearmanr: -0.0833181369461044


...

{'En-Mc-30.Txt': 0.19243564207032,
 'En-Men-Tr-3K.Txt': 0.16889764424023349,
 'En-Mturk-287.Txt': 0.3655361482279775,
 'En-Mturk-771.Txt': 0.15968136285355408,
 'En-Rg-65.Txt': 0.0853308655265738,
 'En-Rw-Stanford.Txt': -0.04517928499062085,
 'En-Simlex-999.Txt': 0.0778555007608628,
 'En-Simverb-3500.Txt': 0.04384192288639279,
 'En-Verb-143.Txt': 0.31843412903887086,
 'En-Ws-353-All.Txt': 0.1750721985231492,
 'En-Ws-353-Rel.Txt': 0.20352634032254005,
 'En-Ws-353-Sim.Txt': 0.1762724941243856,
 'En-Yp-130.Txt': 0.06965448815135929}
Epoch 1 | Loss: 4.007859196198468| spearmanr: 0.0778555007608628



 99%|█████████▉| 7302/7370 [25:11<01:54,  1.69s/it]

{'En-Mc-30.Txt': 0.13551805779599999,
 'En-Men-Tr-3K.Txt': 0.13838539837552571,
 'En-Mturk-287.Txt': 0.3385543122709399,
 'En-Mturk-771.Txt': 0.16258088374112423,
 'En-Rg-65.Txt': 0.11330733150956578,
 'En-Rw-Stanford.Txt': -0.019866581942378864,
 'En-Simlex-999.Txt': 0.057187672804623293,
 'En-Simverb-3500.Txt': 0.03195668920657461,
 'En-Verb-143.Txt': 0.23287346535635897,
 'En-Ws-353-All.Txt': 0.1411257497349259,
 'En-Ws-353-Rel.Txt': 0.16252583346212096,
 'En-Ws-353-Sim.Txt': 0.18464260645967995,
 'En-Yp-130.Txt': 0.10729914694957374}
Epoch 1 | Loss: 3.995485972744229| spearmanr: 0.057187672804623293



100%|██████████| 1/1 [25:25<00:00, 1525.29s/it]

{'En-Mc-30.Txt': 0.09905138042543998,
 'En-Men-Tr-3K.Txt': 0.1439096279099246,
 'En-Mturk-287.Txt': 0.38359101254644906,
 'En-Mturk-771.Txt': 0.1158539179814825,
 'En-Rg-65.Txt': 0.07774178637995878,
 'En-Rw-Stanford.Txt': -0.012910144040946793,
 'En-Simlex-999.Txt': 0.026184719134607792,
 'En-Simverb-3500.Txt': 0.02374412829137752,
 'En-Verb-143.Txt': 0.33542800850759086,
 'En-Ws-353-All.Txt': 0.15030707854828557,
 'En-Ws-353-Rel.Txt': 0.21738623994710485,
 'En-Ws-353-Sim.Txt': 0.17126727826078836,
 'En-Yp-130.Txt': 0.09678746216069298}
Epoch 1 | Loss: 3.98387914075942| spearmanr: 0.026184719134607792
Model trained.
Output saved.


# 5 - Testing

The **word2box** project provides several test datasets inside the `word2box/data/similarity_datasets` folder. They can be used to test the model on the word similarity task.

To test the model, we first need to load the `best_model.ckpt` file. Then, we have to call the `model_eval` method of the previously created `trainer`. You can see the implementation of this method in **section 3.4**.



## 5.1 Load the model

In [ ]:
MODEL_FILENAME = 'best_model.ckpt'

model_path = SAVED_MODELS_DIR + '/' + MODEL_FILENAME

# Load the model's checkpoint
model.load_checkpoint(model_path)

# If you're using a GPU, move the model to the GPU
if torch.cuda.is_available():
    model = model.cuda()

## 5.2 Evaluate the model on word similarity task

In [ ]:
trainer.model_eval(model=model)

{'En-Mc-30.Txt': 0.06258470305488,
 'En-Men-Tr-3K.Txt': 0.1599207869380512,
 'En-Mturk-287.Txt': 0.3404155169628823,
 'En-Mturk-771.Txt': 0.16082983148945287,
 'En-Rg-65.Txt': 0.0859654923541653,
 'En-Rw-Stanford.Txt': 0.03820726082554591,
 'En-Simlex-999.Txt': 0.101616594761613,
 'En-Simverb-3500.Txt': -0.0034523562617459263,
 'En-Verb-143.Txt': 0.23299413195613689,
 'En-Ws-353-All.Txt': 0.1524565619017167,
 'En-Ws-353-Rel.Txt': 0.17931554988870288,
 'En-Ws-353-Sim.Txt': 0.13537891426090742,
 'En-Yp-130.Txt': 0.06658094481768495}


{'En-Ws-353-Sim.Txt': 0.13537891426090742,
 'En-Rg-65.Txt': 0.0859654923541653,
 'En-Yp-130.Txt': 0.06658094481768495,
 'En-Mturk-771.Txt': 0.16082983148945287,
 'En-Men-Tr-3K.Txt': 0.1599207869380512,
 'En-Mturk-287.Txt': 0.3404155169628823,
 'En-Verb-143.Txt': 0.23299413195613689,
 'En-Mc-30.Txt': 0.06258470305488,
 'En-Ws-353-All.Txt': 0.1524565619017167,
 'En-Rw-Stanford.Txt': 0.03820726082554591,
 'En-Simlex-999.Txt': 0.101616594761613,
 'En-Ws-353-Rel.Txt': 0.17931554988870288,
 'En-Simverb-3500.Txt': -0.0034523562617459263}

#6 - Conclusions

This notebook provides an in-depth analysis of the **word2box** algorithm. It also provides ready-to-run code to train and test the model. To train the model, you can use the dataset provided by the *word2box* repository or a dataset that I have previously preprocessed. To further explore the algorithm, the training parameters can be tuned in different ways.